In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
input_size = 784
hidden_size = 500
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [5]:
train_dataset = torchvision.datasets.MNIST(root='D:/', 
                                           train=True, 
                                           transform=transforms.ToTensor(),  
                                           download=False)

test_dataset = torchvision.datasets.MNIST(root='D:/', 
                                          train=False, 
                                          transform=transforms.ToTensor())


In [6]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [7]:
class NeuralNet(nn.Module):
    
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [8]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_step}], Loss: {loss.item():.4f}')

Epoch [1/5], Step [100/600], Loss: 0.3779
Epoch [1/5], Step [200/600], Loss: 0.2244
Epoch [1/5], Step [300/600], Loss: 0.2400
Epoch [1/5], Step [400/600], Loss: 0.2753
Epoch [1/5], Step [500/600], Loss: 0.0659
Epoch [1/5], Step [600/600], Loss: 0.2706
Epoch [2/5], Step [100/600], Loss: 0.0324
Epoch [2/5], Step [200/600], Loss: 0.1644
Epoch [2/5], Step [300/600], Loss: 0.1836
Epoch [2/5], Step [400/600], Loss: 0.1155
Epoch [2/5], Step [500/600], Loss: 0.0307
Epoch [2/5], Step [600/600], Loss: 0.1257
Epoch [3/5], Step [100/600], Loss: 0.0777
Epoch [3/5], Step [200/600], Loss: 0.0488
Epoch [3/5], Step [300/600], Loss: 0.0529
Epoch [3/5], Step [400/600], Loss: 0.0666
Epoch [3/5], Step [500/600], Loss: 0.0792
Epoch [3/5], Step [600/600], Loss: 0.0550
Epoch [4/5], Step [100/600], Loss: 0.0769
Epoch [4/5], Step [200/600], Loss: 0.0680
Epoch [4/5], Step [300/600], Loss: 0.0852
Epoch [4/5], Step [400/600], Loss: 0.0995
Epoch [4/5], Step [500/600], Loss: 0.0105
Epoch [4/5], Step [600/600], Loss:

In [11]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total}%')

Accuracy of the network on the 10000 test images: 97.77%


In [12]:
torch.save(model.state_dict(), 'model.ckpt')